In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from sklearn.ensemble import RandomForestRegressor
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# --- CSV ファイル取得 ---
script_dir = os.path.dirname(__file__)
csv_files = glob.glob(os.path.join(script_dir, "*.csv"))
if not csv_files:
    raise FileNotFoundError("同じフォルダにCSVファイルが見つかりません。")

# --- カラーバー範囲（例） ---
color_ranges = {
    "SiO2 Etching depth": (0, 1700),
    "SiO2 Etching rate": (0, 600),
    "Mask Etching depth": (0, 1400),
    "Mask Selectivity ratio": (1, 6),
    "Vpp": (0, 1000),
    "Vdc": (-800, 0)
}

for file_path in csv_files:
    print(f"読み込み中: {file_path}")
    df = pd.read_csv(file_path, encoding="shift_jis", header=None)

    # --- 対象値名 ---
    full_name = str(df.iloc[0,0])
    target_name = full_name.split("/", 1)[1].strip() if "/" in full_name else full_name.strip()
    print(f"対象値: {target_name}")

    # --- Coil, Platen, z の抽出 ---
    coil_x = df.iloc[1,1:].astype(float).values
    platen_y = df.iloc[3:,0].astype(float).values
    z_values = df.iloc[3:,1:].astype(float).values

    x_list, y_list, z_list = [], [], []
    for i, y_val in enumerate(platen_y):
        for j, x_val in enumerate(coil_x):
            x_list.append(x_val)
            y_list.append(y_val)
            z_list.append(z_values[i, j])

    x = np.array(x_list)
    y = np.array(y_list)
    z = np.array(z_list)

    # --- NaN 除去 ---
    mask = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(z)
    x = x[mask]
    y = y[mask]
    z = z[mask]

    # --- ランダムフォレストで学習 ---
    X = np.column_stack((x, y))
    y_target = z
    model = RandomForestRegressor(n_estimators=500, random_state=42)
    model.fit(X, y_target)

    # --- グリッド作成 ---
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    Xi, Yi = np.meshgrid(xi, yi)
    Zi = model.predict(np.column_stack((Xi.ravel(), Yi.ravel()))).reshape(Xi.shape)

    # --- 下限値を0に制限（Vdcは除外） ---
    if target_name != "Vdc":
        Zi = np.clip(Zi, 0, None)

    # --- vmin, vmax を設定 ---
    vmin, vmax = color_ranges.get(target_name, (Zi.min(), Zi.max()))

    # --- 3Dプロット描画 ---
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    surf = ax.plot_surface(Xi, Yi, Zi, cmap=cm.viridis, edgecolor='none', alpha=0.9)
    ax.scatter(x, y, z, c='red', s=50, edgecolors="black", label="Measured")

    ax.set_xlabel("Coil [W]")
    ax.set_ylabel("Platen [W]")
    ax.set_zlabel(target_name)
    ax.set_title(f"3D Mapping of {target_name}")

    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label=target_name)
    plt.legend()
    plt.show()

    # --- 保存 ---
    safe_name = re.sub(r'[\\/:*?"<>|]', '_', str(target_name))
    output_file = f"{safe_name}_3D_prediction.png"
    output_path = os.path.join(script_dir, output_file)
    fig.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"グラフを保存しました: {output_path}")